In [ ]:
bucket = 'bucket-mldemo-dev-20200505120005202200000002'
 
# Define IAM role
import re


In [ ]:
import boto3
boto3_session=boto3.Session(profile_name='ml-demo')

In [ ]:
from sagemaker.session import Session
from sagemaker import get_execution_role

sagemaker_session=Session(boto3_session)

try:
    role = get_execution_role()
except:
    iam=boto3_session.client('iam')
    response=iam.get_role(
        RoleName='mldemo-dev-Sagemaker-Execution-Role'
    )
    role=response['Role']['Arn']
role

In [ ]:
output_location = 's3://{}/output'.format(bucket)
output_location

In [ ]:
from sagemaker.predictor import RealTimePredictor
linear_predictor=RealTimePredictor('mldemo-dev',sagemaker_session)

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [ ]:
from s3fs.core import S3FileSystem
s3fs = S3FileSystem(anon=False,session=boto3_session)

import pandas as pd
df=pd.read_csv(s3fs.open('s3://%s/transformed/all_data.csv' % bucket),header=None)

df

In [ ]:
evaluation_data_df=df[round(len(df)*0.85+1):].copy().reset_index(drop=True)
evaluation_data_df

In [ ]:
data=','.join([str(x) for x in evaluation_data_df.loc[0].tolist()[0:605]])
linear_predictor.predict(data)

In [ ]:
def eval_func(row):
    csv=','.join(row.tolist()[0:605])
    result=linear_predictor.predict(csv)
    return result
results=evaluation_data_df.astype(str).apply(eval_func,axis=1)

In [ ]:
predicted_labels=[1 if i['predictions'][0]['predicted_label']==1.0 else 0 for i in results]

In [ ]:
score=[i['predictions'][0]['score'] for i in results]

In [42]:
evaluation_data_df['predicted_labels']=predicted_labels
evaluation_data_df['score']=score
evaluation_data_df

,0,1,2,3,4,5,6,7,8,9,...,598,599,600,601,602,603,604,605,predicted_labels,score
0,-18.0,18.0,19.08,234.97,-124.53,-2.420000,-2.286667,-2.210,-2.190,-2.220,...,0.13,0.10,0.13,0.21,0.24,0.190000,0.200000,1,0,0.248378
1,-18.0,18.0,19.14,235.03,-124.47,-2.226667,-2.150000,-2.130,-2.160,-2.170,...,0.16,0.19,0.27,0.30,0.25,0.260000,0.030000,1,0,0.253305
2,-18.0,18.0,19.08,234.97,-124.53,-2.286667,-2.210000,-2.190,-2.220,-2.230,...,0.10,0.13,0.21,0.24,0.19,0.200000,-0.030000,1,0,0.251887
3,-18.0,18.0,19.07,234.96,-124.54,-2.220000,-2.200000,-2.230,-2.240,-2.020,...,0.12,0.20,0.23,0.18,0.19,-0.040000,-0.040000,1,0,0.253339
4,-18.0,18.0,19.26,235.15,-124.35,-2.030000,-2.010000,-2.040,-2.050,-1.830,...,0.31,0.39,0.42,0.37,0.38,0.150000,0.150000,1,0,0.257864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,-18.0,18.0,-17.14,198.75,-160.75,-22.860000,-22.880000,-23.285,-23.320,-23.240,...,0.61,0.60,0.61,0.75,0.69,0.940000,0.620000,0,0,0.246594
1506,-18.0,18.0,-16.87,199.02,-160.48,-22.610000,-23.015000,-23.050,-22.970,-22.895,...,0.87,0.88,1.02,0.96,1.21,0.890000,0.373333,0,0,0.242551
1507,-18.0,18.0,-17.14,198.75,-160.75,-22.880000,-23.285000,-23.320,-23.240,-23.165,...,0.60,0.61,0.75,0.69,0.94,0.620000,0.103333,0,0,0.236395
1508,-18.0,18.0,-16.74,199.15,-160.35,-22.885000,-22.920000,-22.840,-22.765,-22.590,...,1.01,1.15,1.09,1.34,1.02,0.503333,0.265000,0,0,0.249592


In [43]:
evaluation_data_df[evaluation_data_df['predicted_labels']==1]

,0,1,2,3,4,5,6,7,8,9,...,598,599,600,601,602,603,604,605,predicted_labels,score
171,-18.0,18.0,19.35,235.24,-124.26,14.805,14.3300,14.5700,14.65,14.640000,...,0.246667,0.645,0.70,0.69,0.406667,0.265,0.265000,1,1,0.637307
283,-18.0,18.0,8.16,224.05,-135.45,2.130,2.7400,3.1275,3.11,3.180000,...,1.820000,1.940,2.12,2.04,2.080000,2.040,1.140000,1,1,0.641139
284,-18.0,18.0,8.15,224.04,-135.46,2.120,2.7300,3.1175,3.10,3.170000,...,1.810000,1.930,2.11,2.03,2.070000,2.030,1.130000,1,1,0.640851
285,-18.0,18.0,8.13,224.02,-135.48,2.710,3.0975,3.0800,3.15,3.186667,...,1.910000,2.090,2.01,2.05,2.010000,1.110,0.156667,1,1,0.636939


In [44]:
most_certain=evaluation_data_df.query('score>=0.6')
most_certain.sort_values('score',ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,598,599,600,601,602,603,604,605,predicted_labels,score
283,-18.0,18.0,8.16,224.05,-135.45,2.130000,2.7400,3.127500,3.110000,3.180000,...,1.820000,1.940000,2.120000,2.040,2.080000,2.040,1.140000,1,1,0.641139
284,-18.0,18.0,8.15,224.04,-135.46,2.120000,2.7300,3.117500,3.100000,3.170000,...,1.810000,1.930000,2.110000,2.030,2.070000,2.030,1.130000,1,1,0.640851
171,-18.0,18.0,19.35,235.24,-124.26,14.805000,14.3300,14.570000,14.650000,14.640000,...,0.246667,0.645000,0.700000,0.690,0.406667,0.265,0.265000,1,1,0.637307
285,-18.0,18.0,8.13,224.02,-135.48,2.710000,3.0975,3.080000,3.150000,3.186667,...,1.910000,2.090000,2.010000,2.050,2.010000,1.110,0.156667,1,1,0.636939
167,-18.0,18.0,19.35,235.24,-124.26,15.173333,14.7900,14.805000,14.330000,14.570000,...,0.365000,0.395000,0.246667,0.645,0.700000,0.690,0.406667,1,0,0.635822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,-18.0,18.0,17.43,233.32,-126.18,14.435000,14.3800,14.320000,14.800000,14.970000,...,-1.880000,-1.763333,-1.510000,-0.510,-0.220000,-0.080,-0.070000,1,0,0.601138
193,-18.0,18.0,19.31,235.20,-124.30,16.202500,16.3100,16.500000,16.475000,16.480000,...,0.500000,0.080000,0.250000,0.080,0.120000,-0.180,-0.010000,1,0,0.601022
140,-18.0,18.0,22.31,238.20,-121.30,18.377500,19.1175,18.785000,19.020000,18.630000,...,0.130000,0.110000,0.080000,0.060,-0.300000,-0.250,-0.120000,1,0,0.600559
291,-18.0,18.0,7.75,223.64,-135.86,2.700000,2.7700,2.806667,2.656667,2.337500,...,1.630000,1.670000,1.630000,0.730,-0.223333,-0.070,0.050000,1,0,0.600380


In [45]:
# Importing the metrics package from sklearn library
from sklearn import metrics
# Creating the confusion matrix
cm = metrics.confusion_matrix(most_certain[605], most_certain['predicted_labels'])
# Assigning columns names
cm_df = pd.DataFrame(cm, 
            columns = ['Predicted Negative', 'Predicted Positive'],
            index = ['Actual Negative', 'Actual Positive'])
# Showing the confusion matrix
cm_df

,Predicted Negative,Predicted Positive
Actual Negative,0,0
Actual Positive,67,4


In [ ]:
most_certain.query('predicted_labels==1').describe()

In [ ]:
most_certain.query('predicted_labels==0').describe()